In [1]:
%load_ext autoreload
%autoreload 2
%cd /home/xux/Desktop/Enzyme_design/Savi_design/Savinase-engineering

/home/xux/Desktop/Enzyme_design/Savi_design/Savinase-engineering


# Load R1 and R2 data

In [2]:
import pandas as pd

df_R1_merged =  pd.read_csv('data/round1/wet-lab/mcs_uniq_join_duplicated-yh-top_merged2_updated.csv', index_col=0)
df_R1_merged.rename(columns={'mean_act': 'activity_final'}, inplace=True)

df_R1_merged.head(2)

,id,library,chip_loc,sequence,savinase_mutations,n_muts,activity,is_in_mc,mut_not_in,is_dup,mean_activity,n_dup,act_max,act_min,act_min_max_diff,dup_locs,dup_activities,activity_final,checked_act
chip_loc_id,,,,,,,,,,,,,,,,,,,
MC3-8-22,MC3-8-22-MP-F2_B10,Library 3,MC3-8-B-10,AQTVPWGISRVQAPAAHNRGLTGPGVKVAVLDTGISTHPDLKIRGG...,S3T/S24P/N43K/A98S/Y209L/T213R/N218S/T224A/S256K,9,5.607833,1,NaN,0,2.769094,NaN,NaN,NaN,NaN,NaN,NaN,2.769094,4.207822
MC4-1-43,rf 1 MC4-1-43-MP-F2_D05,Library 4,MC4-1-D-07,AQVVPWGISRVQAPAAHNRGLTGSGVKVAVLDTGISTHPDLNIRGG...,S3V/N76D/N204S/Y209L/T213E/N218S/T224A/S256N,8,3.041900,1,NaN,0,3.029921,NaN,NaN,NaN,NaN,NaN,NaN,3.029921,3.239346


In [3]:
df_R2_merged =  pd.read_csv('data/round2/wet-lab/R2-variants-merged.csv')
df_R2_merged.rename(columns={'aa_seq': 'sequence', 'activity': 'activity_final'}, inplace=True)
df_R2_merged.tail()

,sequence,activity_final,mutations_BPN_pos
220,AQSVPWGISRVQAPAAHNRGLTGSGVKVAVLDTGISTHPDLNIRGG...,2.562587,N76D/A88S/A98S/S99N/P131S/A194S/Y209L/T213A/N2...
221,AQVVPWGISRVQAPAAHNRGLTGSGVKVAVLDTGISTHPDLEIRGG...,2.543812,S3V/N43E/A88S/A98S/S99N/P131S/N185Q/A194S/Y209...
222,AQVVPWGISRVQAPAAHNRGLTGSGVKVAVLDTGISTHPDLRIRGG...,2.537806,S3V/N43R/A88S/A98S/S99N/P131S/N185Q/Y209L/T213...
223,AQSVPWGISRVQAPAAHNRGLTGSGVKVAVLDTGISTHPDLNIRGG...,2.966268,N76D/A88S/A98S/S99N/P131S/N185D/A194S/Y209L/T2...
224,AQVVPWGISRVQAPAAHNRGLTGSGVKVAVLDTGISTHPDLKIRGG...,2.730018,S3V/N43K/A88S/A98S/S99N/P131S/Y209L/T213A/N218...


In [4]:
df_merged = pd.concat([df_R1_merged[['sequence', 'activity_final']], df_R2_merged[['sequence', 'activity_final']]], ignore_index=True)
df_merged.to_csv('data/round2/wet-lab/data_all.csv', index=False)

# Get new mutations


In [5]:
# Load all mutations in >2.5x improvement mutants
savinase = 'AQSVPWGISRVQAPAAHNRGLTGSGVKVAVLDTGI-STHPDLNIRGGASFVPGEPST-QDGNGHGTHVAGTIAALNNSIGVLGVAPSAELYAVKVLGASGSGSVSSIAQGLEWAGNNGMHVANLSLGSPSPSATLEQAVNSATSRGVLVVAASGNSG----AGSISYPARYANAMAVGATDQNNNRASFSQYGAGLDIVAPGVNVQSTYPGSTYASLNGTSMATPHVAGAAALVKQKNPSWSNVQIRNHLKNTATSLGSTNLYGSGLVNAEAATR'

def get_mutations(ref, seq):
    mutations = []
    seq_pos = 0
    for i, r in enumerate(ref):
        if r == '-':
            continue
        if r != seq[seq_pos]:
            mutations.append(f'{r}{i+1}{seq[seq_pos]}')
        seq_pos += 1
        
    return '/'.join(mutations)

df_mutants_improved = df_merged[df_merged['activity_final'] > 2.5].copy()
df_mutants_improved['mutatiosn_BPN_pos'] = df_mutants_improved['sequence'].apply(lambda x: get_mutations(savinase, x))

df_mutants_improved.head()

,sequence,activity_final,mutatiosn_BPN_pos
0,AQTVPWGISRVQAPAAHNRGLTGPGVKVAVLDTGISTHPDLKIRGG...,2.769094,S3T/S24P/N43K/A98S/Y209L/T213R/N218S/T224A/S256K
1,AQVVPWGISRVQAPAAHNRGLTGSGVKVAVLDTGISTHPDLNIRGG...,3.029921,S3V/N76D/N204S/Y209L/T213E/N218S/T224A/S256N
2,AQSVPWGISRVQAPAAHNRGLTGPGVKVAVLDTGISTHPDLNIRGG...,2.771272,S24P/A98S/Q206L/Y209H/T213R/N218S/T224A/S256K/...
3,AQTVPWGISRVQAPAAHNRGLTGSGVKVAVLDTGISTHPDLNIRGG...,3.021990,S3T/A98S/P131T/Y209L/T213A/N218S/T224A/S256Y/E...
4,AQTVPWGISRVQAPAAHNRGLTGCGVKVAVLDTGISTHPDLRIRGG...,2.701799,S3T/S24C/N43R/A98S/P131T/A152S/A169G/Y209H/T21...


In [6]:
improved_mutations = set()
for m in df_mutants_improved['mutatiosn_BPN_pos']:
    improved_mutations.update([mut[1:] for mut in m.split('/')])

improved_mutations = sorted(improved_mutations)
print(len(improved_mutations), improved_mutations)

100 ['108V', '114V', '117S', '12K', '130P', '131D', '131G', '131S', '131T', '133P', '142T', '144Y', '151P', '151T', '152S', '166T', '169G', '172T', '175V', '17Y', '185D', '185H', '185Q', '185Y', '188R', '194S', '199I', '201S', '204D', '204K', '204S', '205M', '206L', '206P', '208I', '209F', '209H', '209L', '213A', '213E', '213R', '213Y', '215D', '217M', '218R', '218S', '223S', '224A', '230V', '234A', '234F', '243Y', '245K', '24C', '24P', '24R', '256C', '256G', '256K', '256N', '256R', '256Y', '258E', '259I', '265A', '265G', '265P', '265T', '271Q', '272E', '273E', '38I', '38N', '3A', '3G', '3L', '3N', '3R', '3T', '3V', '3Y', '43A', '43E', '43K', '43R', '52L', '52S', '62S', '74T', '76D', '79V', '85P', '85T', '88S', '88T', '89K', '98G', '98L', '98S', '99N']


# Exclude publication mutations

In [7]:
# Exclude mutations in publications
df_publications = pd.read_csv('data/round1/activity/Activity-AAPF-25-BPN.csv')

# df_publications.head()
pub_mutations = set()
for m in df_publications['Mutations']:
    if m != 'wild-type' and  m != 'wild-type ':
        pub_mutations.update([mut[1:] for mut in m.strip().split('/')])

# Extend Savinase activity mutations:
pub_savi_mutations = set(['V149I', 'R249E', 'W106K', 'V149I', 'W106K', 'M124L', 'M124L', 'V149I', 'W106K', 'M124L', 'W106K', 'V149I', 'W106K', 'M124L', 'M124L', 'V149I', 'W106K', 'M124L', 'V68C', 'V104S', 'L124M', 'P129S', 'S130G', 'P131E', 'S132', 'A133S', 'T134L', 'L135I', 'G97P'])

# Add 'S101'saturated mutations
pub_savi_mutations.update(['S101' + m for m in ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'T', 'V', 'W', 'Y']])

# pub_savi_mutations
pub_mutations.update([mut[1:] for mut in pub_savi_mutations])
improved_remove_pub_mutations = set(improved_mutations) - set(pub_mutations)
print(len(improved_remove_pub_mutations), improved_remove_pub_mutations)

86 {'209H', '272E', '24R', '38I', '234F', '265T', '74T', '88S', '166T', '85P', '206L', '114V', '194S', '38N', '98L', '245K', '12K', '256N', '79V', '144Y', '185Y', '259I', '98G', '199I', '265P', '98S', '43E', '151P', '213R', '205M', '256R', '185D', '256C', '62S', '273E', '142T', '88T', '108V', '169G', '243Y', '265A', '151T', '3N', '256K', '258E', '3Y', '185Q', '3A', '172T', '43K', '204S', '234A', '213A', '133P', '209L', '85T', '131G', '43A', '131T', '204K', '256G', '218R', '3V', '215D', '43R', '52S', '3L', '24P', '17Y', '131S', '130P', '204D', '201S', '208I', '271Q', '3R', '3G', '117S', '52L', '188R', '175V', '230V', '265G', '89K', '99N', '185H'}


## Exclude patent mutations

In [8]:
# Exclude mutations in patents

df_patents = pd.read_excel('data/round2/patents/Patent-restricts-v4.3.xlsx', sheet_name='Patents&Rules')
df_patents.to_csv('data/round2/patents/Patent-restricts-v4.3.csv', index=False)
df_patents.head(2)


,PatentID,Region,Status,Applicant,CoreSites,Rules,Rule_comments,SeqSimilarity,Identity (%),Authorize_status,Effect,Wild-type,Numbering,EffectDesribe,Group
0,CN102648273B,中国,授权,诺维信,其相对亲本枯草杆菌蛋白酶的取代为S9R、A15T、V68A、N218D和Q245R，以及一种...,X9R&A15T&V68A&N218D&Q245R&(G61E|N76D|A98S|S99G...,没有序列一致性的限定，为封闭式的,NaN,100.0,Authorized,NaN,NaN,NaN,NaN,NaN
1,CN102648277B,中国,授权,诺维信,"S9R,A15T,V68A,H120N,N218D,Q245R；\nS9R,A15T,V68...",X9R&A15T&V68A&H120N&N218D&Q245R;S9R&A15T&V68A&...,封闭式限定到具体突变组合，只要与各组合不完全相同即可,NaN,100.0,Authorized,NaN,NaN,NaN,NaN,NaN


In [9]:
import re
import numpy as np
from loguru import logger

def get_muts_from_rule(rule):
    if type(rule) == float and np.isnan(rule):
        return

    muts = re.split('\&|\;|\(|\)|\||\+|\]|\[', rule)
    # check the format of the rule by checking the mutation positions
    for mut in muts:
        if mut != '' and '{' not in mut and '^' not in mut and not mut.isdigit():
            try:
                pos = int(mut[1:-1])
            except:
                logger.warning(f"error/special mut {mut} in rule {rule}")
    
    muts = set([mut[1:] for mut in muts if mut != ''])
    
    # handle the case of multiple mutation choices at the same position
    uniq_muts = set()
    for mut in muts:
        if '{' in mut:
            try:
                pos = int(mut.split('{')[0])
                choices = mut.split('{')[1][:-1].split(',')
                for choice in choices:
                    uniq_muts.add(f'{pos}{choice}')
            except:
                logger.error(f"error in mut {mut}")
        else:
            uniq_muts.add(mut)

    return list(uniq_muts - set(['']))

patent_mutations = set()
for rule in df_patents['Rules']:
    muts = get_muts_from_rule(rule)
    if muts:
        patent_mutations.update(muts)
# ignore speciall mutations if int(mut[:-1]) cannot cast into a int type
patent_mutations = set([mut for mut in patent_mutations if mut[:-1].isdigit()])
patent_mutations = sorted([f"{int(mut[:-1])}{mut[-1]}" for mut in patent_mutations])
print(len(patent_mutations), patent_mutations)

2025-09-30 12:21:22.049 | WARNING  | __main__:get_muts_from_rule:16 - error/special mut G97GG in rule X9{R,K,H}&X15{G,A,S,T,M}&X68{G,A,S,T,M}&X218{D,S,G,V}&X245{R,K,H}&(X61{D,E}|X62{D,E}|X76{D,E}|G97GG|X98{G,S}|X99G|X101G|X120{V,Q,D}|X131{T,S}|X137H|X194P|X228V|X230V|X261D)
2025-09-30 12:21:22.051 | WARNING  | __main__:get_muts_from_rule:16 - error/special mut G097PP in rule X87N&Q109Q&G118D&S128L&P129Q&S130A&S188S&T213E&N248R;S87N&Q109R&G118V&S128L&P129Q&S130A&S188D&T213R&N248D;S87N&Q109Q&G118V&S128L&P129Q&S130A&S188S&T213T&N248N;S87N&Q109R&G118V&S128L&P129Q&S130A&S188D&T213E&N248N;S87N&Q109R&G118V&S128L&P129Q&S130A&S188D&T213E&N248R;S87N&Q109R&G118V&S128L&P129Q&S130A&S188D&T213T&N248R;S87N&Q109R&G118V&S128L&P129Q&S130A&S188S&T213T&N248R;S87R&Q109R&G118V&S128L&P129Q&S130A&S188D&T213E&N248R;S87R&Q109Q&G118R&S128L&P129Q&S130A&S188D&T213T&N248R;S87R&Q109R&G118R&S128L&P129Q&S130A&S188D&T213T&N248R;S87N&Q109Q&G118R&S128L&P129Q&S130A&S188D&T213E&N248R;S87N&Q109R&G118V&S128L&P129Q&S130A&S188

2025-09-30 12:21:22.058 | WARNING  | __main__:get_muts_from_rule:16 - error/special mut I35ID in rule X215{K,R,Q,N,S,T}&(3+X3T|X4I|X9E|I35ID|X43R|X76D|X99{D,F}|X101{E,L}|X103{A,T}|X104I|X120D|X160S|X195E|X205I|X206L|X209W|X235L|X259D|X261W|X262E)
2025-09-30 12:21:22.059 | WARNING  | __main__:get_muts_from_rule:16 - error/special mut I35ID in rule X215{K,R,Q,N,S,T}&(3+X3T|X4I|X9E|I35ID|X43R|X76D|X99{D,F}|X101{E,L}|X103{A,T}|X104I|X120D|X160S|X195E|X205I|X206L|X209W|X235L|X259D|X261W|X262E)


534 ['101A', '101D', '101E', '101F', '101G', '101H', '101K', '101L', '101M', '101N', '101Q', '101R', '101T', '101V', '101Y', '103A', '103G', '103N', '103P', '103T', '104F', '104I', '104L', '104T', '104Y', '105D', '105E', '105K', '106A', '106W', '107G', '107I', '107N', '107S', '107T', '107V', '107W', '109D', '109G', '109I', '109K', '109L', '109N', '109Q', '109R', '109T', '109V', '10A', '114V', '115R', '115T', '115W', '116A', '116L', '116V', '117E', '118D', '118R', '118V', '118W', '120D', '120I', '120K', '120N', '120Q', '120R', '120T', '120V', '1213E', '1213T', '121F', '122H', '124M', '126F', '126R', '127I', '127Q', '127R', '127S', '127T', '128A', '128I', '128L', '128R', '128T', '129D', '129E', '129Q', '12D', '12E', '130A', '131-', '131S', '131T', '132H', '132N', '133D', '133E', '133P', '136Q', '137H', '141F', '141H', '141N', '141Y', '143W', '144T', '144V', '144W', '144Y', '145H', '146S', '147W', '149C', '149N', '14T', '150L', '156D', '158E', '159D', '159E', '15A', '15G', '15K', '15M', '

In [10]:
# Load other patents from round1
df_patents_R1 = pd.read_csv('data/round1/hotspots/subtilisin_patents_n_muts.csv')

df_patents_R1.head()

patent_mutations_R1 = set()
for mut in df_patents_R1['mutations']:
    if mut != 'Savinase':
        patent_mutations_R1.update([m[1:] for m in mut.split('/')])

patent_mutations_R1 = sorted(patent_mutations_R1)
print(len(patent_mutations_R1), patent_mutations_R1)

563 ['100C', '100GE', '100L', '100S', '100aA', '100bG', '100cS', '100dG', '101G', '101P', '101R', '101SA', '101SK', '101T', '102D', '102S', '103A', '103D', '103E', '103H', '103L', '103Q', '103Y', '104I', '104L', '104M', '104N', '104R', '104S', '104T', '104Y', '105D', '105G', '106A', '106D', '106E', '106F', '106G', '106I', '106L', '106M', '106T', '107M', '107T', '107V', '108S', '108T', '108V', '10H', '10K', '111F', '111I', '111V', '114V', '116D', '116S', '118D', '119I', '11A', '11M', '120D', '120E', '120G', '120I', '120K', '120N', '120Q', '120R', '120S', '120V', '120Y', '121A', '123D', '124C', '124I', '125A', '126F', '126I', '127E', '128A', '128G', '128I', '128N', '129D', '129E', '129PSN', '129S', '129T', '12D', '12E', '130*', '130AT', '130P', '130SP', '130T', '130aG', '131*', '131A', '131AA', '131C', '131D', '131F', '131H', '131L', '131M', '131Q', '131R', '131S', '131T', '131W', '132G', '133ASA', '133D', '133E', '133S', '134A', '134S', '137A', '137D', '137E', '137H', '137L', '138D', '1

In [11]:
improved_remove_pub_pat_mutations = improved_remove_pub_mutations - set(patent_mutations) - set(patent_mutations_R1)
improved_remove_pub_pat_mutations = sorted(improved_remove_pub_pat_mutations)
print(len(improved_remove_pub_pat_mutations), improved_remove_pub_pat_mutations)

41 ['117S', '12K', '131G', '142T', '151P', '151T', '166T', '17Y', '185H', '185Y', '194S', '199I', '201S', '204K', '205M', '208I', '234A', '234F', '243Y', '256K', '258E', '259I', '265A', '265P', '265T', '271Q', '272E', '273E', '38I', '38N', '3G', '3N', '3R', '43K', '52L', '52S', '74T', '79V', '85P', '88S', '89K']


# Sort the variants by the occourences in the improved mutants

In [12]:
improved_remove_pub_pat_mutations

dict_improved_new_muts = [ (int(mut[:-1]), mut[-1]) for mut in improved_remove_pub_pat_mutations]
df_improved_new_muts = pd.DataFrame(dict_improved_new_muts, columns=['pos', 'mut'])

def count_mutation_fequency(mutation, mutant_mutations):
    return sum([1 if mutation in mutant else 0 for mutant in mutant_mutations])


mutant_mutations = df_mutants_improved['mutatiosn_BPN_pos'].apply(lambda x: [ mut[1:] for mut in x.split('/')])
df_improved_new_muts['frequency'] = df_improved_new_muts.apply(lambda x: count_mutation_fequency(f"{x['pos']}{x['mut']}", mutant_mutations), axis=1)

df_improved_new_muts.sort_values('frequency', ascending=False, inplace=True)
df_improved_new_muts.to_csv('data/round2/wet-lab/improved_new_muts.csv', index=False)

print(len(df_improved_new_muts))
df_improved_new_muts.head()

41


,pos,mut,frequency
25,271,Q,170
39,88,S,164
10,194,S,114
22,265,A,95
33,43,K,93


In [13]:
import numpy as np

def convert_into_savinase_position(positions):
    """ Convert from BPN positions to savinase positions """
    uniq_pos = list(set(positions))
    uniq_pos_sorted = np.sort(uniq_pos).tolist()

    # Convert the positions into savinase positions
    savinase = 'AQSVPWGISRVQAPAAHNRGLTGSGVKVAVLDTGI*STHPDLNIRGGASFVPGEPST*QDGNGHGTHVAGTIAALNNSIGVLGVAPSAELYAVKVLGASGSGSVSSIAQGLEWAGNNGMHVANLSLGSPSPSATLEQAVNSATSRGVLVVAASGNSG*AGS***ISYPARYANAMAVGATDQNNNRASFSQYGAGLDIVAPGVNVQSTYPGSTYASLNGTSMATPHVAGAAALVKQKNPSWSNVQIRNHLKNTATSLGSTNLYGSGLVNAEAATR'

    sav_pos = []
    idx_diff = 0
    for i, aa in enumerate(savinase):
        if aa == '*': idx_diff += 1

        if i+1 in uniq_pos_sorted:
            sav_pos.append(i+1 - idx_diff)
    
    # restore the original order
    pos_mapping = dict(zip(uniq_pos_sorted, sav_pos))
    sav_pos = [pos_mapping[p] for p in positions]
    return sav_pos

# convert_into_savinase_position([57])

In [14]:

df_improved_new_muts_pos = pd.DataFrame(df_improved_new_muts.pos.unique(), columns=['pos'])
df_improved_new_muts_pos['new_muts'] = [df_improved_new_muts[df_improved_new_muts.pos == pos].mut.tolist() for pos in df_improved_new_muts_pos.pos]

df_improved_new_muts_pos['mut_freqs'] = [df_improved_new_muts[df_improved_new_muts.pos == pos].frequency.tolist() for pos in df_improved_new_muts_pos.pos]

df_improved_new_muts_pos['total_freqs'] = df_improved_new_muts_pos['mut_freqs'].apply(sum)
df_improved_new_muts_pos['savi_pos'] = convert_into_savinase_position(df_improved_new_muts_pos['pos'].to_list())


df_improved_new_muts_pos.sort_values('pos', inplace=True)
df_improved_new_muts_pos.reset_index(drop=True, inplace=True)
df_improved_new_muts_pos.to_csv('data/round2/wet-lab/improved_new_muts_pos.csv', index=False)

print(len(df_improved_new_muts_pos))

df_improved_new_muts_pos

32


,pos,new_muts,mut_freqs,total_freqs,savi_pos
0,3,"[N, R, G]","[8, 1, 1]",10,3
1,12,[K],[1],1,12
2,17,[Y],[2],2,17
3,38,"[I, N]","[1, 1]",2,37
4,43,[K],[93],93,42
5,52,"[S, L]","[1, 1]",2,51
6,74,[T],[1],1,72
7,79,[V],[1],1,77
8,85,[P],[1],1,83
9,88,[S],[164],164,86
